In [27]:
import numpy as np
import scipy as sc
import pandas as pd
import sklearn as sk
from importlib import reload
import matplotlib.pyplot as plt

import ReadFromDB as RDB
from Authenticator import read_auth

# ReadFromSB functions: 
##    get_static_data()
#     read_weather
#     station
#     create_station_dictionary
#     station_dict_row

In [33]:
# reload after changes have been made
reload(RDB)
dir(RDB)[8:]

['create_station_dictionary',
 'get_static_data',
 'pd',
 'read_auth',
 'read_weather',
 'station',
 'station_dict_row']

In [62]:
static_data = RDB.get_static_data()
station_numbers = list(static_data['number'].astype(int))
static_data.head(2)

,number,name,address,latitude,longitude
0,2,BLESSINGTON STREET,Blessington Street,53.356769,-6.268140
1,3,BOLTON STREET,Bolton Street,53.351182,-6.269859


In [38]:
RDB.read_weather(5).head(2)

,apparentTemperature,cloudCover,dewPoint,humidity,icon,nearestStormBearing,nearestStormDistance,ozone,precipIntensity,precipProbability,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,12.42,0.75,5.72,0.64,partly-cloudy-day,304.0,206.0,283.67,0.0,0.0,1032.57,Mostly Cloudy,12.42,1551095170,2.0,9.46,166.0,8.45,1.82
1,12.52,0.75,5.74,0.63,partly-cloudy-day,304.0,206.0,283.62,0.0,0.0,1032.57,Mostly Cloudy,12.52,1551095426,2.0,9.46,166.0,8.45,1.82


In [49]:
RDB.station(5).head(2)

,status,bike_stands,available_bike_stands,available_bikes,last_updated
0,OPEN,40,38,2,1.550592e+12
1,OPEN,40,37,3,1.550593e+12


In [67]:
stations_data = RDB.create_station_dictionary(station_numbers[0])

In [71]:
stations_data[2]

,status,bike_stands,available_bike_stands,available_bikes,last_updated
0,OPEN,20,19,1,1.550593e+12
1,OPEN,20,20,0,1.550593e+12
2,OPEN,20,19,1,1.550594e+12
3,OPEN,20,19,1,1.550594e+12
4,OPEN,20,20,0,1.550595e+12
5,OPEN,20,20,0,1.550596e+12
6,OPEN,20,19,1,1.550596e+12
7,OPEN,20,19,1,1.550597e+12
8,OPEN,20,19,1,1.550598e+12
9,OPEN,20,20,0,1.550598e+12
